# Generating Word Embeddings - Lab

## Introduction

In this lab, you'll learn how to generate word embeddings by training a Word2Vec model, and then embedding layers into deep neural networks for NLP!

## Objectives

You will be able to:

- Train a Word2Vec model and transform words into vectors 
- Obtain most similar words by using methods associated with word vectors 


## Getting Started

In this lab, you'll start by creating your own word embeddings by making use of the Word2Vec model. Then, you'll move onto building neural networks that make use of **_Embedding Layers_** to accomplish the same end-goal, but directly in your model. 

As you've seen, the easiest way to make use of Word2Vec is to import it from the [Gensim Library](https://radimrehurek.com/gensim/). This model contains a full implementation of Word2Vec, which you can use to begin training immediately. For this lab, you'll be working with the [News Category Dataset from Kaggle](https://www.kaggle.com/rmisra/news-category-dataset/version/2#_=_).  This dataset contains headlines and article descriptions from the news, as well as categories for which type of article they belong to.

Run the cell below to import everything you'll need for this lab. 

In [2]:
!pip install gensim

  Created wheel for smart-open: filename=smart_open-4.1.0-py3-none-any.whl size=106210 sha256=d05d71e9c0b38a75c3247eaeb9b9cee1afd64a09105ac541529efbac4800cbe1
  Stored in directory: c:\users\andrew\appdata\local\pip\cache\wheels\08\1f\ba\87200f08f3aaf24e33dc6f2547a6b7eecda4f5dfbb585d1f5e
Successfully built smart-open
  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.13
    Uninstalling Cython-0.29.13:
      Successfully uninstalled Cython-0.29.13


In [3]:
import pandas as pd
import numpy as np
np.random.seed(0)
from gensim.models import Word2Vec
from nltk import word_tokenize

Now, import the data. The data is stored in the file `'News_Category_Dataset_v2.json'`.  This file is compressed, so that it can be more easily stored in a GitHub repo. **_Make sure to unzip the file before continuing!_**

In the cell below, use the `read_json()` function from Pandas to read the dataset into a DataFrame. Be sure to include the parameter `lines=True` when reading in the dataset!

Once you've imported the data, inspect the first few rows of the DataFrame to see what your data looks like. 

In [5]:
df = pd.read_json('News_Category_Dataset_v2.json', lines=True)
df

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26
...,...,...,...,...,...,...
200848,TECH,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,"Reuters, Reuters",https://www.huffingtonpost.com/entry/rim-ceo-t...,Verizon Wireless and AT&T are already promotin...,2012-01-28
200849,SPORTS,Maria Sharapova Stunned By Victoria Azarenka I...,,https://www.huffingtonpost.com/entry/maria-sha...,"Afterward, Azarenka, more effusive with the pr...",2012-01-28
200850,SPORTS,"Giants Over Patriots, Jets Over Colts Among M...",,https://www.huffingtonpost.com/entry/super-bow...,"Leading up to Super Bowl XLVI, the most talked...",2012-01-28
200851,SPORTS,Aldon Smith Arrested: 49ers Linebacker Busted ...,,https://www.huffingtonpost.com/entry/aldon-smi...,CORRECTION: An earlier version of this story i...,2012-01-28


## Preparing the Data

Since you're working with text data, you need to do some basic preprocessing including tokenization. Notice from the data sample that two different columns contain text data -- `headline` and `short_description`. The more text data your Word2Vec model has, the better it will perform. Therefore, you'll want to combine the two columns before tokenizing each comment and training your Word2Vec model. 

In the cell below:

* Create a column called `'combined_text'` that consists of the data from the `'headline'` column plus a space character (`' '`) plus the data from the `'short_description'` column 
* Use the `'combined_text'` column's `.map()` method and pass in `word_tokenize`. Store the result returned in `data` 

In [8]:
import nltk

In [9]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Andrew\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [10]:
df['combined_text'] = df['headline'] + ' ' +  df['short_description']
data = df['combined_text'].map(word_tokenize)

Inspect the first 5 items in `data` to see how everything looks. 

In [11]:
data[:5]

0    [There, Were, 2, Mass, Shootings, In, Texas, L...
1    [Will, Smith, Joins, Diplo, And, Nicky, Jam, F...
2    [Hugh, Grant, Marries, For, The, First, Time, ...
3    [Jim, Carrey, Blasts, 'Castrato, ', Adam, Schi...
4    [Julianna, Margulies, Uses, Donald, Trump, Poo...
Name: combined_text, dtype: object

Notice that although the words are tokenized, they are still in the same order they were in as headlines. This is important, because the words need to be in their original order for Word2Vec to establish the meaning of them. Remember that for a Word2Vec model you can specify a  **_Window Size_** that tells the model how many words to take into consideration at one time. 

If your window size was 5, then the model would start by looking at the words "Will Smith joins Diplo and", and then slide the window by one, so that it's looking at "Smith joins Diplo and Nicky", and so on, until it had completely processed the text example at index 1 above. By doing this for every piece of text in the entire dataset, the Word2Vec model learns excellent vector representations for each word in an **_Embedding Space_**, where the relationships between vectors capture semantic meaning (recall the vector that captures gender in the previous "king - man + woman = queen" example you saw).

Now that you've prepared the data, train your model and explore a bit!

## Training the Model

Start by instantiating a Word2Vec Model from `gensim`. 

In the cell below:

* Create a `Word2Vec` model and pass in the following arguments:
    * The dataset we'll be training on, `data`
    * The size of the word vectors to create, `size=100`
    * The window size, `window=5`
    * The minimum number of times a word needs to appear in order to be counted in  the model, `min_count=1` 
    * The number of threads to use during training, `workers=4`

In [12]:
model = Word2Vec(data, size=100, window=5, min_count=1, workers=4)

Now, that you've instantiated Word2Vec model, train it on your text data. 

In the cell below:

* Call the `.train()` method on your model and pass in the following parameters:
    * The dataset we'll be training on, `data`
    * The `total_examples`  of sentences in the dataset, which you can find in `model.corpus_count` 
    * The number of `epochs` you want to train for, which we'll set to `10`

In [13]:
model.train(data, total_examples=model.corpus_count, epochs=10)

(55560909, 67352790)

Great! You now have a fully trained model! The word vectors themselves are stored in the `Word2VecKeyedVectors` instance, which is stored in the `.wv` attribute. To simplify this, restore this object inside of the variable `wv` to save yourself some keystrokes down the line. 

In [14]:
wv = model.wv

## Examining Your Word Vectors

Now that you have a trained Word2Vec model, go ahead and explore the relationships between some of the words in the corpus! 

One cool thing you can use Word2Vec for is to get the most similar words to a given word. You can do this by passing in the word to `wv.most_similar()`. 

In the cell below, try getting the most similar word to `'Texas'`.

In [15]:
wv.most_similar('Texas')

[('Louisiana', 0.8282762169837952),
 ('Oregon', 0.8263369202613831),
 ('Pennsylvania', 0.8041567802429199),
 ('Maryland', 0.8019928336143494),
 ('Georgia', 0.7912709712982178),
 ('Colorado', 0.7863339781761169),
 ('Arkansas', 0.7805790901184082),
 ('Oklahoma', 0.7795277237892151),
 ('Florida', 0.7758452296257019),
 ('Arizona', 0.7747176885604858)]

Interesting! All of the most similar words are also states. 

You can also get the least similar vectors to a given word by passing in the word to the `.most_similar()` method's `negative` parameter. 

In the cell below, get the least similar words to `'Texas'`.

In [16]:
wv.most_similar(negative='Texas')

[('Headstrong', 0.3986521363258362),
 ('Strategist™', 0.38716447353363037),
 ('Note-Taking', 0.3684598207473755),
 ('uh-oh', 0.36634424328804016),
 ('Admon', 0.3639875650405884),
 ('keenest', 0.3591766953468323),
 ('Animal-rescue', 0.35257598757743835),
 ('Recognise', 0.35180529952049255),
 ('Anti-Nepotism', 0.34771716594696045),
 ('Coalesce', 0.347602516412735)]

This seems like random noise. It is a result of the way Word2Vec is computing the similarity between word vectors in the embedding space. Although the word vectors closest to a given word vector are almost certainly going to have similar meaning or connotation with your given word, the word vectors that the model considers 'least similar' are just the word vectors that are farthest away, or have the lowest cosine similarity. It's important to understand that while the closest vectors in the embedding space will almost certainly share some level of semantic meaning with a given word, there is no guarantee that this relationship will hold at large distances. 

You can also get the vector for a given word by passing in the word as if you were passing in a key to a dictionary. 

In the cell below, get the word vector for `'Texas'`.

In [17]:
wv['Texas']

array([-0.26418093, -0.10025296,  0.6468161 , -1.0266713 , -0.05463576,
        1.6864667 , -1.6881979 ,  2.9474692 ,  0.7710111 , -1.4782914 ,
        1.0382665 , -1.149242  ,  0.5487077 , -0.54677993, -1.4249873 ,
       -0.30367127,  0.41260636, -1.5036243 , -0.0582994 , -2.188825  ,
        2.2505903 , -1.02683   ,  0.7124602 ,  1.2638088 ,  0.01043367,
        2.662286  ,  1.339989  , -0.22998543, -1.3495338 ,  0.76691496,
        1.1239955 , -0.03623512, -2.3602405 , -0.5486733 , -0.07658301,
        0.6650936 , -2.259091  , -0.888135  ,  0.56584793,  0.7638811 ,
        1.9220796 ,  4.262951  , -2.639461  ,  0.31144416,  0.31560704,
        1.7732409 ,  0.5723531 ,  1.9934632 ,  0.19614306, -0.08300821,
        2.1103384 ,  0.982696  ,  1.3808758 , -2.5341892 ,  0.9955025 ,
        0.15261695,  0.8691894 ,  0.92449987,  1.2445215 ,  1.1370057 ,
        0.8458842 , -0.8918661 , -0.5977484 , -1.7203224 ,  0.4582495 ,
       -1.1442006 ,  0.49569374, -0.6631837 , -0.60713834,  0.50

Now get all of the word vectors from the object at once. You can find these inside of `wv.vectors`. Try it out in the cell below.  

In [18]:
wv.vectors

array([[ 7.9285741e-01, -1.7917719e+00, -1.6284227e-01, ...,
         5.8812320e-01, -2.0170374e+00, -6.8371135e-01],
       [ 4.8119086e-01, -1.3508680e+00,  1.7930050e+00, ...,
         9.7574836e-01, -1.8689010e-01, -1.3929983e+00],
       [ 1.2015302e+00, -1.6838251e+00,  2.0724473e+00, ...,
         1.1717026e+00, -1.1169306e+00,  1.3093191e+00],
       ...,
       [-8.3218999e-02,  1.1126479e-01, -4.3469228e-02, ...,
        -4.8084699e-02,  5.2414257e-02, -4.8375934e-02],
       [-3.9782811e-02, -4.5427319e-04, -1.2500748e-02, ...,
         6.4532191e-02,  7.7136923e-03, -1.0395837e-01],
       [-7.7556498e-02,  9.8313294e-02, -1.1638018e-02, ...,
        -1.1338174e-02,  2.4908114e-02, -5.2171879e-02]], dtype=float32)

As a final exercise, try to recreate the _'king' - 'man' + 'woman' = 'queen'_ example previously mentioned. You can do this by using the `.most_similar()` method and translating the word analogies into an addition/subtraction formulation (as shown above). Pass the original comparison, which you are calculating a difference between, to the negative parameter, and the analogous starter you want to apply the same transformation to, to the `positive` parameter.

Do this now in the cell below. 

In [19]:
wv.most_similar(positive=['king', 'woman'], negative=['man'])

[('revival', 0.6300833225250244),
 ('title', 0.6087403893470764),
 ('queen', 0.5842036008834839),
 ('symbol', 0.5701772570610046),
 ('reminiscent', 0.569121778011322),
 ('princess', 0.565441370010376),
 ('character', 0.5647317171096802),
 ('crown', 0.5625712871551514),
 ('sculpture', 0.5620573163032532),
 ('remix', 0.5597060322761536)]

As you can see from the output above, your model isn't perfect, but 'Queen' and 'Princess' are still in the top 5. As you can see from the other word in top 5, 'reminiscent' -- your model is far from perfect. This is likely because you didn't have enough training data. That said, given the small amount of training data provided, the model still performs remarkably well! 

In the next lab, you'll reinvestigate transfer learning, loading in the weights from an open-sourced model that has already been trained for a very long time on a massive amount of data. Specifically, you'll work with the GloVe model from the Stanford NLP Group. There's not really any benefit from training the model ourselves, unless your text uses different, specialized vocabulary that isn't likely to be well represented inside an open-source model.

## Summary

In this lab, you learned how to train and use a Word2Vec model to create vectorized word embeddings!